# Import libs

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymssa import MSSA
from src.asc_itmo_lab.chaos_indic import HurstTraj, NoiseFactor, DimEmb, max_lyapunov, fourier_harmonic_count, ks_entropy_partition
from tqdm import tqdm
import os
import seaborn as sns
import time


from sklearn.metrics import mean_absolute_percentage_error
from lightgbm import LGBMRegressor

# EDA

In [ ]:
m4_datasets = pd.read_csv('/Users/roman14/Downloads/Reservoir-Computing-Investigation/M4 Competition Dataset/m4_info.csv')
m4_datasets.head()

In [ ]:
m4_datasets.info()

In [ ]:
m4_datasets['category'].value_counts(normalize=True)

In [ ]:
m4_datasets['SP'].value_counts()

In [ ]:
m4_datasets.groupby(by=['category','SP']).count()['M4id']

In [ ]:
def list_train_csv_files(directory='.'):
    """
    Lists all files in the given directory that end with '-train.csv' and returns their count and names.
    
    Parameters:
        directory (str): Path to the directory to search. Defaults to current directory.
        
    Returns:
        tuple: (count of matching files, list of matching filenames)
    """
    files = os.listdir(directory)
    train_files = [f for f in files if f.endswith('-train.csv')]
    return len(train_files), train_files


len_train_files, files = list_train_csv_files(os.getcwd() + '/M4 Competition Dataset')
print(f"Found {len_train_files} '-train.csv' files:")
for file in files:
    print(file)


# Get statistics

In [ ]:
# Find series with length at least 1000 points
ts_id = []
time_series_df = pd.DataFrame()

for file in files:
    df = pd.read_csv(os.path.join(os.getcwd() + '/M4 Competition Dataset', file)).T

    if df.shape[0] > 1000:
        df.reset_index(inplace=True, drop=True)
        df.columns = df.iloc[0]          # first row becomes header
        df.drop(df.index[0], inplace=True)
        notna=df.notna().sum()
        ts = notna.keys()[np.where(notna.values>1000)]
        ts_id.append(ts.to_list())
        filtered_series = df[ts].dropna()
        time_series_df = pd.concat([time_series_df, filtered_series], axis=1)

time_series_df.reset_index(drop=True, inplace=True)

In [ ]:
time_series_df = time_series_df.dropna()

In [ ]:
ts_ids = np.concatenate(ts_id).tolist()

In [ ]:
m4_datasets_cleaned = m4_datasets[m4_datasets['M4id'].isin(ts_ids)]

In [ ]:
# Define desired proportions and counts for 650 samples
proportions = m4_datasets_cleaned['category'].value_counts(normalize=True).to_dict()
total_to_sample = 1501
desired_counts = {cat: int(round(p * total_to_sample)) for cat, p in proportions.items()}

# Sample by category
selected_frames = []
for cat, count in desired_counts.items():
    pool = m4_datasets_cleaned[m4_datasets_cleaned['category'] == cat]
    selected = pool.sample(n=count, random_state=23)
    selected_frames.append(selected)

selected_info = pd.concat(selected_frames).reset_index(drop=True)

In [ ]:
m4_datasets_cleaned['category'].value_counts()

In [ ]:
m4_datasets_cleaned['SP'].value_counts()

In [ ]:
# def collect_series(file_list, min_obs=850, target_n=650):
#     """
#     Iterate over CSVs in file_list, selecting columns with at least min_obs
#     non-NaN values, until we have target_n series. Returns a DataFrame whose
#     columns are the selected series.
    
#     file_list : list of filenames (strings)
#     min_obs    : minimum non-NaN count per series
#     target_n   : total series to collect
#     """
#     collected = pd.DataFrame()  # will hold selected columns
#     collected_count = 0
    
#     for fname in file_list:
#         if collected_count >= target_n:
#             break
        
#         # 1) Load and promote row 0 to header
#         df = pd.read_csv(os.path.join(os.getcwd() + '/M4 Competition Dataset', fname))
#         df = df.T
#         df.reset_index(inplace=True, drop=True)
#         df.columns = df.iloc[0]          # first row becomes header
#         df.drop(df.index[0], inplace=True)
        
#         # 2) For each column, count non-NaNs
#         for col in df.columns:
#             if collected_count >= target_n:
#                 break
            
#             non_nans = df[col].notna().sum()
#             if non_nans >= min_obs:
#                 # add this column to collected (align on index length)
#                 # pad shorter series with NaN if needed
#                 series = df[col]
#                 if len(series) < min_obs:
#                     continue  # shouldn't happen, but guard
#                 # if collected is empty, initialize with index
#                 if collected_count == 0:
#                     collected = pd.DataFrame(index=range(len(series)))
#                 # pad/truncate to collected.index length
#                 L = len(collected)
#                 s = series.reset_index(drop=True)
#                 if len(s) > L:
#                     s = s.iloc[:L]
#                 elif len(s) < L:
#                     s = s.reindex(range(L))
#                 collected[col] = s
#                 collected_count += 1
        
#     print(f"Collected {collected_count} series out of requested {target_n}.")
#     return collected

# result_df = collect_series(files, min_obs=1000, target_n=1500)
# print("Resulting DataFrame shape:", result_df.shape)

In [ ]:
# time_series_df = result_df.dropna()
# time_series_df.info()

In [ ]:
for col in time_series_df.columns:
    try:
        time_series_df[col] = pd.to_numeric(time_series_df[col], errors='raise')
    except ValueError:
        # skip columns that aren’t pure numbers
        pass

In [ ]:
time_series_df = time_series_df[selected_info['M4id'].to_list()]
time_series_df.info()

In [ ]:
# time_series_df.to_csv('time_series.csv', header=True, index=False)

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(time_series_df['W19'])
plt.ylabel('Value')
plt.xlabel('Timestep')
plt.title('W19 time-series')
plt.grid()

In [ ]:
time_series_df.columns

In [ ]:
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
plt.plot(time_series_df['W19'])
plt.ylabel('Value')
plt.xlabel('Timestep')
plt.title('W19 time-series')
plt.grid()

plt.subplot(1, 2, 2)
plt.plot(time_series_df['D2452'])
plt.ylabel('Value')
plt.xlabel('Timestep')
plt.title('D2452 time-series')
plt.grid()

plt.tight_layout()
plt.show()

## Measures

In [ ]:
estimations = []

for series_name in tqdm(time_series_df.columns):
    # Only process if enough data points
    if len(time_series_df[series_name]) < 1:
        continue
    
    # Compute metrics
    _, _, hurst_exponent, _ = HurstTraj(time_series_df[series_name])
    noise = NoiseFactor(time_series_df[series_name])
    _, corr_dim, _ = DimEmb(time_series_df[series_name])
    lyap_max = max_lyapunov(time_series_df[series_name].values)
    ks_ent = ks_entropy_partition(time_series_df[series_name])
    fourier_count = fourier_harmonic_count(time_series_df[series_name])
    
    estimations.append({
        'series': series_name,
        'hurst': hurst_exponent,
        'noise_factor': noise,
        'corr_dim': corr_dim,
        'lyap_max': lyap_max,
        'ks_entropy': ks_ent,
        'fourier_harmonics': fourier_count
    })

measures_df = pd.DataFrame(estimations)

In [ ]:
measures_df.info()

In [ ]:
measures_df.head()

In [ ]:
measures_df.describe()

In [ ]:
measures_df.to_csv('measures_df.csv', header=True)

In [ ]:
measures_df.columns

In [ ]:
columns = ['hurst', 'noise_factor', 'corr_dim', 'lyap_max', 'ks_entropy', 'fourier_harmonics']

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 8))

for i, col in enumerate(columns):
    row = i // 3
    col_num = i % 3
    ax = axes[row, col_num]
    
    sns.kdeplot(data=measures_df[col], ax=ax, fill=True, color='skyblue', alpha=0.5, linewidth=1.5)
    
    median_val = measures_df[col].median()
    mean_val = measures_df[col].mean()
    
    ax.axvline(median_val, color='red', linestyle='--', linewidth=1.5, label=f'Median: {median_val:.2f}')
    ax.axvline(mean_val, color='green', linestyle='-', linewidth=1.5, label=f'Mean: {mean_val:.2f}')
    
    ax.legend(loc='upper right')
    ax.set_title(f'Distribution of {col}', fontsize=12, pad=10)
    ax.set_xlabel('Value', fontsize=10)
    ax.set_ylabel('Density', fontsize=10)
    
    ax.tick_params(axis='both', which='major', labelsize=8)

if len(columns) < 6:
    for i in range(len(columns), 6):
        fig.delaxes(axes.flatten()[i])

plt.tight_layout(pad=3.0)

In [ ]:
columns = ['hurst', 'noise_factor', 'corr_dim', 'lyap_max', 'ks_entropy', 'fourier_harmonics']

# Создаем фигуру с несколькими subplots
fig, axes = plt.subplots(nrows=len(columns), ncols=1, figsize=(10, 2 * len(columns)))

# Проходим по всем колонкам и строим KDE для каждой
for i, col in enumerate(columns):
    ax = axes[i]
    
    # Строим KDE
    sns.kdeplot(data=measures_df[col], ax=ax, fill=True, alpha=0.5, linewidth=1.5)
    
    # Вычисляем медиану и среднее
    median_val = measures_df[col].median()
    mean_val = measures_df[col].mean()
    
    # Добавляем вертикальные линии для медианы и среднего
    ax.axvline(median_val, color='red', linestyle='--', linewidth=1, label=f'Median: {median_val:.2f}')
    ax.axvline(mean_val, color='green', linestyle='-', linewidth=1, label=f'Mean: {mean_val:.2f}')
    
    # Добавляем легенду и заголовок
    ax.legend()
    ax.set_title(f'Distribution of {col}')
    ax.set_xlabel('')
    ax.set_ylabel('Density')

plt.tight_layout()
plt.show()

# Predictions

In [ ]:
def train_test(x, test_step):
    X_train = x[:-test_step-1]
    Y_train = x[1:-test_step]
    y_test = x[-test_step:]

    return X_train, Y_train, y_test

## LightGBM

In [ ]:
import warnings
from sklearn.exceptions import DataConversionWarning

# ignore just the specific DataConversionWarning
warnings.filterwarnings("ignore", category=DataConversionWarning)

# ignore the “X does not have valid feature names” UserWarning
warnings.filterwarnings(
    "ignore",
    message="X does not have valid feature names"
)

l_res_lgbm = []
time_spent_list = []

for ts in tqdm(measures_df['series']):
    start_time = time.time()

    lgbm = LGBMRegressor(random_state=23, verbosity=-1)
    l_res_ts = []
    for i in range(15, -1, -1):
        X_train, Y_train, y_test = train_test(time_series_df[ts], test_step=15+i)
        lgbm = lgbm.fit(X_train.values.reshape(-1,1), Y_train.values.ravel())
        
        cur = Y_train.values[-1]
        preds = []
        for j in range(15):
            cur = lgbm.predict(cur.reshape(1, -1)).reshape(-1, 1)[0]
            preds.append(cur)
        
        l_res_ts.append(mean_absolute_percentage_error(y_test[:15], preds)*100)
    
    l_res_lgbm.append(np.mean(l_res_ts))
    end_time = time.time()  # Засекаем время окончания обработки серии
    time_spent = end_time - start_time  # Вычисляем общее время выполнения
    time_spent_list.append(time_spent)  # Добавляем в список

measures_df['mape_lightgbm'] = l_res_lgbm
measures_df['time_spent_lightgbm'] = time_spent_list 

In [ ]:
plt.figure(figsize=(10,4))
sns.boxplot(x=l_res_lgbm, orient="x")
plt.title('Box-Plot of LightGBM MAPE predictions')
plt.xlim((0,10))
plt.ylabel('MAPE, %')

In [ ]:
round(np.mean(l_res_lgbm),2)

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(measures_df['hurst'], measures_df['mape_lightgbm'])
plt.xlabel('Hurst exponent')
plt.ylabel('LightGBM MAPE')
plt.ylim((0,20))
plt.grid()

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(measures_df['ks_entropy'], measures_df['mape_lightgbm'])
plt.xlabel('KS entropy')
plt.ylabel('LightGBM MAPE')
plt.ylim((0,20))
plt.grid()

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(measures_df['noise_factor'], measures_df['mape_lightgbm'])
plt.xlabel('Noise measure (big-better)')
plt.ylabel('LightGBM MAPE')
plt.ylim((0,50))
plt.grid()

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(measures_df['fourier_harmonics'], measures_df['mape_lightgbm'])
plt.xlabel('Num of Fourier Harmonics')
plt.ylabel('LightGBM MAPE')
plt.ylim((0,50))
plt.grid()

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(measures_df['corr_dim'], measures_df['mape_lightgbm'])
plt.xlabel('Correlation dimension')
plt.ylabel('LightGBM MAPE')
plt.ylim((0,50))
plt.grid()

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(measures_df['lyap_max'], measures_df['mape_lightgbm'])
plt.xlabel('Lyapunov Exponent')
plt.ylabel('LightGBM MAPE')
plt.ylim((0,50))
plt.xlim((0, 0.2))
plt.grid()

## MSSA

In [ ]:

l_res_mssa = []
time_spent_list = []
for ts in tqdm(measures_df['series']):
    start_time = time.time()
    mssa = MSSA(verbose=False)
    l_res_ts = []
    for i in range(15, -1, -1):
        _, _, y_test = train_test(time_series_df[ts], test_step=15+i)
        mssa.fit(time_series_df[ts][:-15-i])

        preds = mssa.forecast(15).ravel()

        l_res_ts.append(mean_absolute_percentage_error(y_test[:15], preds)*100)
    
    l_res_mssa.append(np.mean(l_res_ts))
    end_time = time.time()  # Засекаем время окончания обработки серии
    time_spent = end_time - start_time  # Вычисляем общее время выполнения
    time_spent_list.append(time_spent)  # Добавляем в список

measures_df['mape_mssa'] = l_res_mssa
measures_df['time_spent_mssa'] = time_spent_list 

In [ ]:
plt.figure(figsize=(10,4))
sns.boxplot(x=measures_df['mape_mssa'], orient="x")
plt.title('Box-Plot of MSSA MAPE')
plt.xlim((0,35))
plt.ylabel('MAPE, %')

In [ ]:
round(np.mean(measures_df['mape_mssa']),2)

## Prophet

In [ ]:
from prophet import Prophet
import warnings

# Silence warnings from Prophet
warnings.filterwarnings('ignore')

def prepare_prophet_df(series, start, freq):
    """Convert your numeric series to a DataFrame compatible with Prophet."""
    df = pd.DataFrame({
        'ds': pd.date_range(start=start, periods=len(series), freq=freq),
        'y': series
    })
    return df

def determine_freq(ts, starting_date):
    if ts.startswith('H'):
        freq ='H'
    elif ts.startswith('D'):
        freq = 'D'
    elif ts.startswith('W'):
        wd = pd.to_datetime(starting_date, dayfirst=True).weekday()
        freq = ['W-MON','W-TUE','W-WED','W-THU','W-FRI','W-SAT','W-SUN'][wd]
    elif ts.startswith('M'):
        if pd.to_datetime(starting_date, dayfirst=True).day > 1:
            freq = 'ME'
        else:
            freq = 'MS'
    elif ts.startswith('Q'):
        if pd.to_datetime(starting_date, dayfirst=True).day > 1:
            freq = 'QE'
        else:
            freq = 'QS'
    elif ts.startswith('Y'):
        if pd.to_datetime(starting_date, dayfirst=True).day > 1:
            freq = 'YE'
        else:
            freq = 'YS'

    return freq

l_res_prophet = []
time_spent_list = []
for ts in tqdm(measures_df['series']):
    start_time = time.time()
    l_res_ts = []

    # Fetch your time series and remove NaNs
    series_data = time_series_df[ts].dropna().values

    for i in range(15, -1, -1):
        test_step = 15 + i
        train_series = series_data[:-test_step]
        y_test = series_data[-test_step:][:15]

        # Prepare data for Prophet
        starting_date = m4_datasets['StartingDate'].loc[m4_datasets['M4id'] == ts].item()
        freq = determine_freq(ts, starting_date)
        train_df = prepare_prophet_df(train_series, start=starting_date, freq=freq)

        # Initialize and fit the Prophet model
        model = Prophet()
        model.fit(train_df)

        # Predict 10 future steps
        future = model.make_future_dataframe(periods=15, freq=freq)  # or your appropriate frequency
        forecast = model.predict(future)

        preds = forecast['yhat'].values[-15:]  # Take last 15 predictions

        # Evaluate predictions
        mape = mean_absolute_percentage_error(y_test, preds) * 100
        l_res_ts.append(mape)

    l_res_prophet.append(np.mean(l_res_ts))
    end_time = time.time()  # Засекаем время окончания обработки серии
    time_spent = end_time - start_time  # Вычисляем общее время выполнения
    time_spent_list.append(time_spent)  # Добавляем в список


measures_df['mape_prophet'] = l_res_prophet
measures_df['time_spent_prophet'] = time_spent_list 

In [ ]:
plt.figure(figsize=(10,4))
sns.boxplot(x=measures_df['mape_prophet'], orient="x")
plt.title('Box-Plot of Prophet MAPE')
plt.xlim((0,25))
plt.ylabel('MAPE, %')

In [ ]:
round(np.mean(measures_df['mape_prophet']),2)

## ESN

In [ ]:
import reservoirpy as rpy
from reservoirpy.nodes import Ridge, Reservoir
rpy.verbosity(0)  # no need to be too verbose here
rpy.set_seed(23)  # make everything reproducible!

units = 100
leak_rate = 0.3
spectral_radius = 0.9
input_scaling = 1.0
connectivity = 0.1
input_connectivity = 0.2
regularization = 1e-8
seed = 23

def reset_esn():
    reservoir = Reservoir(units, input_scaling=input_scaling, sr=spectral_radius,
                        lr=leak_rate, rc_connectivity=connectivity,
                        input_connectivity=input_connectivity, seed=seed)

    readout = Ridge(ridge=regularization)

    return reservoir >> readout

In [ ]:
# l_res_esn = []
# for ts in tqdm(measures_df['series']):
#     l_res_ts = []
#     for i in range(10, -1, -1):
#         X_train, Y_train, y_test = train_test(time_series_df[ts], test_step=10+i)
#         esn = reset_esn()
#         esn = esn.fit(X_train.values.reshape(-1, 1), Y_train.values.reshape(-1, 1))
        
#         cur = Y_train.values.reshape(-1, 1)[-1]
#         preds = []
#         for j in range(10):
#             cur = esn.run(cur).ravel()[0]
#             preds.append(cur)
        
#         l_res_ts.append(mean_absolute_percentage_error(y_test[:10], preds)*100)
    
#     l_res_esn.append(np.mean(l_res_ts))
# measures_df['mape_esn'] = l_res_esn

In [ ]:
# Function to compute log-returns
def to_log_return(x):
    return np.diff(np.log(x))

# Function to reconstruct original prices from log-returns
def from_log_return(last_known_value, log_returns):
    return last_known_value * np.exp(np.cumsum(log_returns))

l_res_esn = []
time_spent_list = []
for ts in tqdm(measures_df['series']):
    start_time = time.time()
    l_res_ts = []
    series_data = time_series_df[ts].dropna().values

    for i in range(15, -1, -1):
        test_step = 15 + i
        
        # split data into train-test
        train_raw = series_data[:-test_step]
        y_test_real = series_data[-test_step:][:15]

        # convert train data to log-returns
        train_log_returns = to_log_return(train_raw)

        # ESN training (assuming reset_esn gives a correctly initialized ESN model)
        esn = reset_esn()
        
        # prepare inputs for ESN (X_train and Y_train as sequential data)
        X_train = train_log_returns[:-1].reshape(-1, 1)
        Y_train = train_log_returns[1:].reshape(-1, 1)
        
        esn.fit(X_train, Y_train)

        # Forecasting
        preds_log_return = []
        current_input = Y_train[-1].reshape(1, -1)

        for j in range(15):
            pred_log_ret = esn.run(current_input).item()
            preds_log_return.append(pred_log_ret)
            current_input = np.array([[pred_log_ret]])

        # reconstruct predictions from log-returns to real data
        preds_real = from_log_return(train_raw[-1], preds_log_return)

        # calculate MAPE on real data
        mape = mean_absolute_percentage_error(y_test_real, preds_real) * 100
        l_res_ts.append(mape)

    l_res_esn.append(np.mean(l_res_ts))
    end_time = time.time()  # Засекаем время окончания обработки серии
    time_spent = end_time - start_time  # Вычисляем общее время выполнения
    time_spent_list.append(time_spent)  # Добавляем в список

    
measures_df['mape_esn'] = l_res_esn
measures_df['time_spent_esn'] = time_spent_list 

In [ ]:
plt.figure(figsize=(10,4))
sns.boxplot(x=measures_df['mape_esn'], orient="x")
plt.title('Box-Plot of ESN MAPE predictions')
plt.xlim((0,10))
plt.ylabel('MAPE, %')

In [ ]:
round(np.mean(l_res_esn),2)

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(measures_df['hurst'], l_res_esn)
plt.xlabel('Hurst exponent')
plt.ylabel('ESN MAPE')
plt.ylim((0,20))
plt.grid()

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(measures_df['ks_entropy'], l_res_esn)
plt.xlabel('KS entropy')
plt.ylabel('ESN MAPE')
plt.ylim((0,20))
plt.grid()

## Fourie ESN

In [ ]:
from sklearn.linear_model import Ridge as Ridge_sklearn
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

class EnhancedESN_FAN:
    def __init__(self,
                 input_dim,
                 reservoir_size=100,
                 spectral_radius=0.9,
                 sparsity=0.1,
                 ridge_alpha=1.0,
                 poly_order=2,
                 fan_dp=5,
                 seed=23):
        """
        input_dim : dimensionality of X (features per timestep)
        fan_dp    : number of Fourier terms (dp in the FAN paper)
        """
        np.random.seed(seed)
        self.reservoir_size = reservoir_size
        self.poly = PolynomialFeatures(degree=poly_order, include_bias=False)
        self.fan_dp = fan_dp

        # placeholders; actual Win/W init happens in fit()
        self.Win = None
        self.W   = None

        self.spectral_radius = spectral_radius
        self.sparsity = sparsity

        self.ridge = Ridge_sklearn(alpha=ridge_alpha)
        self.scaler = StandardScaler()
        self.input_scaler = StandardScaler()

    def initialize_weights(self, X):
        # input‐distribution aware Win
        inp_mean, inp_std = X.mean(axis=0), X.std(axis=0)
        mu, sigma = inp_mean.mean(), inp_std.mean()
        self.Win = np.random.normal(mu, sigma, (self.reservoir_size, X.shape[1] + 1))

        # sparse W with spectral radius
        W = np.random.rand(self.reservoir_size, self.reservoir_size) - 0.5
        W[np.random.rand(*W.shape) > self.sparsity] = 0
        W *= self.spectral_radius / np.max(np.abs(np.linalg.eigvals(W)))
        self.W = W

    def _update(self, state, u):
        # standard ESN update
        pre = self.Win @ np.concatenate(([1], u)) + self.W @ state
        return np.tanh(pre)

    def _fourier_features(self, X):
        # X: (n_samples, n_features)
        # return: (n_samples, 2*fan_dp*n_features)
        # FAN: explicit Fourier series terms sin(2πk x), cos(2πk x)
        features = []
        for k in range(1, self.fan_dp + 1):
            features.append(np.sin(2 * np.pi * k * X))
            features.append(np.cos(2 * np.pi * k * X))
        return np.hstack(features)
    
    # def _fft_features(self, X, n_coeffs):
    #     # X: (T, n) time series
    #     # return first n_coeffs magnitudes per dimension
    #     coeffs = np.abs(fft(X, axis=0))[:n_coeffs]  # shape (n_coeffs, n)
    #     return np.tile(coeffs.flatten(), (X.shape[0], 1))

    def fit(self, X, y):
        """
        X: shape (T, input_dim)
        y: shape (T,) or (T, output_dim)
        """
        # init reservoir
        self.initialize_weights(X)

        # collect reservoir states
        T = X.shape[0]
        states = np.zeros((T, self.reservoir_size))
        state = np.zeros(self.reservoir_size)
        for t in range(T):
            state = self._update(state, X[t])
            states[t] = state

        X_scaled = self.input_scaler.fit_transform(X)
        # polynomial features
        P = self.poly.fit_transform(X_scaled)
        # Fourier analysis network features
        F = self._fourier_features(X_scaled)

        # # combine all
        # H = np.hstack([states, P, F])

        # # scale & fit readout
        # Hs = self.scaler.fit_transform(H)
        # self.ridge.fit(Hs, y)

        H = np.hstack([states, P, F])
        self.scaler.fit(H)
        # prevent division by zero in .transform()
        self.scaler.scale_[self.scaler.scale_ == 0.0] = 1.0
        Hs = self.scaler.transform(H)
        self.ridge.fit(Hs, y)


    def predict(self, X, generative_steps=None):
        """
        If X is (T, input_dim) and generative_steps is None, this is open-loop:
          returns readout(X)
        If generative_steps is an int > 0, we perform:
          - use X[-1] as u0, then recursively predict next u's.
        """
        if generative_steps is None:
            # one‐shot prediction (teacher forcing)
            T = X.shape[0]
            states = np.zeros((T, self.reservoir_size))
            state = np.zeros(self.reservoir_size)
            for t in range(T):
                state = self._update(state, X[t])
                states[t] = state

            P = self.poly.transform(X)
            F = self._fourier_features(X)
            H = np.hstack([states, P, F])
            Hs = self.scaler.transform(H)
            return self.ridge.predict(Hs)

        # generative forecasting
        u = X[-1].copy()      # last observed input
        state = np.zeros(self.reservoir_size)
        preds = []
        for _ in range(generative_steps):
            state = self._update(state, u.reshape(-1))
            # build feature row
            # P = self.poly.transform(u.reshape(1, -1))
            # F = self._fourier_features(u.reshape(1, -1))
            u_scaled = self.input_scaler.transform(u.reshape(1, -1))
            clip = 3.0  # you can tune this (e.g. 3 standard deviations)
            u_scaled = np.clip(u_scaled, -clip, clip)

            P = self.poly.transform(u_scaled)
            F = self._fourier_features(u_scaled)
            h = np.hstack([state, P.ravel(), F.ravel()]).reshape(1, -1)
            h_s = self.scaler.transform(h)
            u_next = self.ridge.predict(h_s)
            preds.append(u_next.ravel())
            u = u_next  # feed back

        return np.vstack(preds)

In [ ]:
l_res_fourie_esn = []
time_spent_list = []
for ts in tqdm(measures_df['series']):
    start_time = time.time()
    l_res_ts = []
    for i in range(15, -1, -1):
        X_train, Y_train, y_test = train_test(time_series_df[ts], test_step=15+i)
        esn_fan = EnhancedESN_FAN(input_dim=1)
        esn_fan.fit(X=X_train.values.reshape(-1,1), y=Y_train.values.reshape(-1,1))
        
        preds = esn_fan.predict(Y_train.values.reshape(-1,1), generative_steps=15)
        
        l_res_ts.append(mean_absolute_percentage_error(y_test[:15], preds)*100)
    
    l_res_fourie_esn.append(np.mean(l_res_ts))
    end_time = time.time()  # Засекаем время окончания обработки серии
    time_spent = end_time - start_time  # Вычисляем общее время выполнения
    time_spent_list.append(time_spent)  # Добавляем в список

measures_df['mape_esn_fan'] = l_res_fourie_esn
measures_df['time_spent_esn_f'] = time_spent_list 

In [ ]:
plt.figure(figsize=(10,4))
sns.boxplot(x=measures_df['mape_esn_fan'], orient="x")
plt.title('Box-Plot of ESN_FAN MAPE')
plt.xlim((0,10))
plt.ylabel('MAPE, %')

In [ ]:
round(np.mean(measures_df['mape_esn_fan']),2)

In [ ]:
measures_df.to_csv('measures_df.csv', header=True)

# Box-plots

In [ ]:
plt.figure(figsize=(10,6))
box = plt.boxplot(measures_df[['mape_esn_fan', 'mape_esn', 'mape_lightgbm', 'mape_prophet',
                                'mape_mssa']].values, labels=['ESN_FT', 'ESN', 'LightGBM', 'Prophet', 'SSA'],
         patch_artist=True, showfliers=False, meanline=True)

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Закрашиваем каждый ящик своим цветом
for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)

for median in box['medians']:
    median.set_color('black')  # Красный цвет медианы
    median.set_linewidth(0.5)  # Можно также изменить толщину

plt.title("Box-Plots of models MAPE")
plt.ylabel('MAPE, %')
plt.xlabel('Models')
plt.ylim((0,35))
plt.grid(axis='y')

In [ ]:
measures_df[['time_spent_esn_f','time_spent_esn','time_spent_lightgbm', 'time_spent_prophet','time_spent_mssa']].mean()

# Clustering

In [ ]:
measures_df.columns

In [ ]:
from sklearn.cluster import KMeans
km=KMeans(n_clusters=2, random_state=23)
km.fit(measures_df[['hurst', 'noise_factor', 'corr_dim', 'lyap_max', 'ks_entropy', 'fourier_harmonics']])
measures_df['label']=km.labels_
m_df2 = measures_df.copy()
m_df2.drop(m_df2[m_df2.mape_esn>100].index, inplace=True)
m_df2.drop(m_df2[m_df2.mape_esn_fan>100].index, inplace=True)
m_df2.describe()

In [ ]:
round(m_df2.groupby('label')[['mape_esn_fan','mape_esn','mape_lightgbm', 'mape_prophet','mape_mssa']].mean(),2)

In [ ]:
m_df2['label'].value_counts()

In [ ]:
m_df2.groupby('label')[['hurst', 'noise_factor', 'corr_dim', 'lyap_max', 'ks_entropy', 'fourier_harmonics']].mean()

In [ ]:
m_df2.drop(m_df2[m_df2.hurst < 0.2].index, inplace=True)

In [ ]:
m_df2['dif']=m_df2['mape_esn_fan']-m_df2['mape_lightgbm']
d0=m_df2[(m_df2.dif<-1)&((m_df2.label==0))]
d1=m_df2[(m_df2.dif>1)&(m_df2.label==0)]
n=len(m_df2[m_df2.label==0])
print('Резервуар лучше: %.1f%%; хуже %.1f%%'%(len(d0)/n*100, len(d1)/n*100), n)
t=['Hurst exp.', 'Noise factor', 'Correlation dim.', 'Lyapunov exp.', 'KS-entropy', 'Fourier harmonics']

plt.figure(figsize=(10,5))
for i, m in enumerate(['hurst', 'noise_factor', 'corr_dim', 'lyap_max', 'ks_entropy', 'fourier_harmonics']):
    plt.subplot(2,3,i+1)
    plt.boxplot([d0[m], d1[m]], positions=[1,2], widths=.7, showfliers=False)
    plt.xticks([1,2],['ESN-FT','LightGBM'])
    plt.ylabel(t[i])
    plt.grid(axis='y')
    print(m, '\t%.2f %.2f'%(d0[m].median(), d1[m].median()))
plt.tight_layout()
# plt.savefig('competition.jpg',transparent=None, dpi=600)
plt.show()

In [ ]:
colors

In [ ]:
import matplotlib.pyplot as plt

t = ['Hurst exp.', 'Noise factor', 'Correlation dim.', 'Lyapunov exp.', 'KS-entropy', 'Fourier harmonics']
colors = ['#1f77b4', '#2ca02c']  # Цвета для ESN-FT и LightGBM

plt.figure(figsize=(10, 5))

for i, m in enumerate(['hurst', 'noise_factor', 'corr_dim', 'lyap_max', 'ks_entropy', 'fourier_harmonics']):
    plt.subplot(2, 3, i + 1)
    box = plt.boxplot([d0[m], d1[m]], positions=[1, 2], widths=.7,
                      patch_artist=True, showfliers=False)

    # Окрашиваем каждый ящик в свой цвет
    for patch, color in zip(box['boxes'], colors):
        patch.set_facecolor(color)

    # Настройка медианы
    for median in box['medians']:
        median.set_color('black')
        median.set_linewidth(0.8)

    plt.xticks([1, 2], ['ESN-FT', 'LightGBM'])
    plt.ylabel(t[i])
    plt.grid(axis='y')
    print(m, '\t%.2f %.2f' % (d0[m].median(), d1[m].median()))

plt.tight_layout()
plt.show()
